In [2]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

In [3]:
class State(TypedDict):
    hello: str
    
graph_builder = StateGraph(State)

In [4]:
def node_first(state: State):
    print("node_first")

def node_second(state: State):
    print("node_second")

def node_third(state: State):
    print("node_third")

In [5]:
graph_builder.add_node("node_first", node_first)
graph_builder.add_node("node_second", node_second)
graph_builder.add_node("node_third", node_third)

graph_builder.add_edge(START, "node_first")
graph_builder.add_edge("node_first", "node_second")
graph_builder.add_edge("node_second", "node_third")
graph_builder.add_edge("node_third", END)

In [8]:
graph = graph_builder.compile()

graph.invoke({})

node_first
node_second
node_third
